In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df.loc[:, 'type'] = df.loc[:, 'type'].replace('defacement', 'malicious')
df.loc[:, 'type'] = df.loc[:, 'type'].replace('phishing', 'malicious')
df.loc[:, 'type'] = df.loc[:, 'type'].replace('malware', 'malicious')

In [4]:
df.head()

,url,type
0,br-icloud.com.br,malicious
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,malicious
4,http://adventure-nicaragua.net/index.php?optio...,malicious


In [5]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
# Preprocess URLs
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['url'])  
sequences = tokenizer.texts_to_sequences(df['url'])
data = pad_sequences(sequences, maxlen=100)

In [7]:
# Convert labels
labels = df['type'].apply(lambda x: 1 if x == 'malicious' else 0).values

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=32, input_length=100))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [9]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
model.fit(data, labels, epochs=1, batch_size=32, validation_split=0.2)



16280/16280 [==============================] - 1941s 119ms/step - loss: 0.0664 - accuracy: 0.9724 - val_loss: 2.9097 - val_accuracy: 0.4167


In [11]:
model.save('url_classification_model.h5')  # Saves the model in HDF5 format

C:\Users\acer\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
def predict_url(model, tokenizer, url):
    # Preprocess the URL
    sequence = tokenizer.texts_to_sequences([url])
    padded_sequence = pad_sequences(sequence, maxlen=100)

    # Make a prediction
    prediction = model.predict(padded_sequence)

    # Interpret the prediction
    if prediction[0][0] > 0.5:
        return "Malicious"
    else:
        return "Benign"




In [13]:
from tensorflow.keras.models import load_model

In [14]:
model_path = 'url_classification_model.h5'
loaded_model = load_model(model_path)

In [15]:
# Example of using the function
input_url = input("Enter a URL to predict: ")
prediction = predict_url(loaded_model, tokenizer, input_url)
print(f"The URL is predicted as: {prediction}")

1/1 [==============================] - 0s 89ms/step
The URL is predicted as: Benign
